In [1]:
import pandas as pd
import numpy as np

### Import data

In [46]:
# maybe use np.where to reduce the data imported

Read

In [3]:
code = pd.read_csv('data/coding7128.tsv', sep='\t')

In [4]:
clinical = pd.read_csv('data/tpp_gp_clinical_subset.csv')

In [5]:
ic10 = pd.read_csv('data/ic10_lookup.csv')

In [6]:
GPic10 = pd.read_csv('data/GP_ic10_lookup.csv')

In [7]:
scripts = pd.read_csv('data/sample_covid19_tpp_gp_scripts.txt')

have a look at the data

In [8]:
code.head()
# CTV3 codes (used by GPs, in dataset clinical) and the corresponding disease and incidences
# code can also be measurements, but will not be used by us

,coding,meaning
0,-1,redacted - potentially sensitive or identifying
1,-2,redacted - rare occupation
2,.....,Read thesaurus
3,.0...,See 0....
4,.01..,See 01...


In [9]:
clinical
# GP dataset from every patient
# each line corresponds to one incident with a date (event_dt)
# eid: patient number 
# code_type: ?
# value: measured value of a test or whatever

,Unnamed: 0,eid,event_dt,code_type,code,value
0,101084316,4392480,31/12/2020,0,XaERu,11.00
1,9359934,1311969,31/12/2020,0,XaN3z,0.80
2,9360007,1311969,31/12/2020,0,44P6.,0.58
3,111269383,4731119,31/12/2020,0,XaIm8,0.00
4,9360002,1311969,31/12/2020,0,44P5.,1.30
...,...,...,...,...,...,...
49995,46497795,2564395,31/12/2019,0,XaFqt,0.00
49996,46497794,2564395,31/12/2019,0,XaIVg,0.00
49997,41639202,2400982,31/12/2019,0,XaEJK,0.00
49998,116371391,4900832,31/12/2019,0,X1004,0.00


In [10]:
ic10
# description of ICD20 codes

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17931,17931,U84.8,U848,DEFAULT,3.0,Resistance to other specified antimicrobial drug,,,,,,,
17932,17932,U84.9,U849,DEFAULT,3.0,Resistance to unspecified antimicrobial drugs,,,,,,,
17933,17933,U85,U85X,DEFAULT,3.0,Resistance to antineoplastic drugs,,,,,,,
17934,17934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
GPic10
#translates CTV3 codes (real_code) to ICD10 (icd10_code)

,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
0,0,123..,Z831,D,C,M,0.0,0.0
1,1,123..,Z830,A,C,M,0.0,0.0
2,2,1231.,Z831,G,C,M,0.0,0.0
3,3,1232.,Z831,G,C,M,0.0,0.0
4,4,1233.,Z831,G,C,M,0.0,0.0
...,...,...,...,...,...,...,...,...
116372,116372,ZVy..,Z769,D,C,C,0.0,0.0
116373,116373,ZVz..,Z769,D,C,C,0.0,0.0
116374,116374,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116375,116375,"Contains information from NHS Digital, licence...",NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
scripts
# all medication patients got from their GP, incl. the COVID-19 vaccine 
# needs to be complete if want to compare vaccinated vs. non-vaccinated, but seems like not all vaccinations registered here but just the ones done by the GP

,Unnamed: 0,eid,issue_date,dmd_code
0,39278003,3743492,2021-03-22,320031002
1,35914327,3508920,2021-03-15,3591211000001105
2,49790799,4472075,2021-05-03,33405511000001102
3,31447155,3192040,2021-03-15,321745007
4,69964306,5879124,2021-01-19,320031002
...,...,...,...,...
99995,62996879,5393814,2021-01-27,320030001
99996,47834233,4333940,2021-01-03,38893411000001105
99997,33831040,3365447,2021-03-31,318052005
99998,12832421,1889162,2021-03-30,321152004


In [13]:
# get few individuals from large dataset -> may try to call only part of scripts dataset, where code == one of covid vaccine

# try only to get lines from scripts dataset where dmd_code== a vaccine 
# use function read_csv. only where irgendwas

print('lines in samplescripts: ', len(scripts))
print('number of subjects in samplescripts: ', len(scripts['eid'].value_counts()))

lines in samplescripts:  100000
number of subjects in samplescripts:  51104


### Preparing the data

#### Adding column with IC10 categories to clinical dataset

In [14]:
# tbd

#### Data Cleaning

In [15]:
# remove NaN in selected columns (esp. Ic10 column in "clinical" dataset)
# remove duplicate entries

### Find who is vaccinated

The meaning of the SNOMED codes ('dmd_code' in the 'scripts' dataset, where medications that the patients got are listed) were found on: https://services.nhsbsa.nhs.uk/dmd-browser/amp/view/159421?ref=YW1wTmFtZT1jb3ZpZCZzZWFyY2hUeXBlPUFNUCZzaG93SW52YWxpZEl0ZW1zPWZhbHNl#ampCodes

Each vaccine has a specific code.

In [16]:
# broadcasting to find individuals with AstraZEneca vaccine
mask_AZ = scripts['dmd_code']== 39114911000001105
scripts[mask_AZ].head()

,Unnamed: 0,eid,issue_date,dmd_code
4561,64814430,5521041,2021-01-20,39114911000001105
5499,16646535,2163004,2021-01-31,39114911000001105
7750,13225838,1916265,2021-02-03,39114911000001105
8932,70996867,5949279,2021-02-22,39114911000001105
9967,4432489,1304258,2021-02-02,39114911000001105


In [26]:
# getting ID's of individuals with any corona vaccine first (compare vaccinates vs. not)
# may look at effects of specific vaccines later
dict_vac = {39326911000001101: 'Moderna', 39230211000001104: 'Janssen', \
            39826711000001101:'Medicago', 39473011000001103: 'Baxter', \
            39114911000001105: 'AstraZeneca', 39115611000001103: 'Pfizer',\
            39373511000001104: 'Valneva'}

for dmd_code in dict_vac:
    mask_vac = scripts['dmd_code']== dmd_code
    print(dict_vac[dmd_code], ':',len(scripts[mask_vac]))

Moderna : 0
Janssen : 0
Medicago : 0
Baxter : 0
AstraZeneca : 23
Pfizer : 7
Valneva : 0


In [28]:
# finding eids of vaccinated people with specific vaccine
id_vac = scripts[mask_AZ]['eid']

In [30]:
# finding events of the vaccinated people, but only events before vaccination found 
# -> may need a more recent dataset
# need to separate clinical events before & after vaccine
mask_vac = clinical['eid'].isin(id_vac)
clinical[mask_vac] 

,Unnamed: 0,eid,event_dt,code_type,code,value,selected disease
31073,34474547,2163004,31/12/2020,0,8HE2.,0.000000,NaN
36034,47865366,2608259,31/12/2019,0,XaIAQ,36.099998,NaN
36086,47865541,2608259,31/12/2019,0,242..,74.000000,NaN
36117,47864684,2608259,31/12/2019,0,XaJK8,0.000000,NaN
36118,47864707,2608259,31/12/2019,0,XaJKJ,0.000000,NaN
36121,47864718,2608259,31/12/2019,0,XaMiI,0.000000,NaN
36122,47864755,2608259,31/12/2019,0,XaMhk,0.000000,NaN
36126,47864891,2608259,31/12/2019,0,XaJKn,0.000000,NaN
36154,47864323,2608259,31/12/2019,0,XE0qn,0.000000,NaN
36167,47864473,2608259,31/12/2019,1,Y0529,0.000000,NaN


# categorize event as before or after COVID-19 vaccination (done by Renée)

### make 'event_date' in 'clinical' be interpreted as a date

In [47]:
clinical.head(10)

,Unnamed: 0,eid,event_dt,code_type,code,value,selected disease
0,101084316,4392480,31/12/2020,0,XaERu,11.00,NaN
1,9359934,1311969,31/12/2020,0,XaN3z,0.80,NaN
2,9360007,1311969,31/12/2020,0,44P6.,0.58,NaN
3,111269383,4731119,31/12/2020,0,XaIm8,0.00,NaN
4,9360002,1311969,31/12/2020,0,44P5.,1.30,NaN
5,111269344,4731119,31/12/2020,0,XaIge,0.00,NaN
6,9359963,1311969,31/12/2020,0,42M..,0.90,NaN
7,9359951,1311969,31/12/2020,0,XE2q4,1.11,NaN
8,9359940,1311969,31/12/2020,0,XE2eD,2.10,NaN
9,9359924,1311969,31/12/2020,0,XaELV,1.59,NaN


In [48]:
from datetime import datetime, timedelta

In [49]:
clinical['event_dt'].unique()
# find out if the other dates are e.g. 06/01/2020 or 6/1/2020

array(['31/12/2020', '31/12/2019'], dtype=object)

In [50]:
example_date = clinical['event_dt'].unique()[0]

In [51]:
example_date_new = datetime.strptime(example_date, '%d/%m/%Y')
example_date_new #2020

datetime.datetime(2020, 12, 31, 0, 0)

In [52]:
example_date_2 = clinical['event_dt'].unique()[1]
example_date_2 

'31/12/2019'

In [53]:
example_date_2_new = datetime.strptime(example_date_2, '%d/%m/%Y')
example_date_2_new #2019

datetime.datetime(2019, 12, 31, 0, 0)

In [54]:
# compare the 2 dates
example_date_new > example_date_2_new # 2020 > 2019 --> True
# recent > older --> True

True

In [55]:
# convert entire dataframe column in datetime format
clinical_copy = clinical # just to not mess up the clinical dataset
clinical_copy['event_dt_date_format'] = pd.to_datetime(clinical_copy['event_dt'])
clinical_copy

,Unnamed: 0,eid,event_dt,code_type,code,value,selected disease,event_dt_date_format
0,101084316,4392480,31/12/2020,0,XaERu,11.00,NaN,2020-12-31
1,9359934,1311969,31/12/2020,0,XaN3z,0.80,NaN,2020-12-31
2,9360007,1311969,31/12/2020,0,44P6.,0.58,NaN,2020-12-31
3,111269383,4731119,31/12/2020,0,XaIm8,0.00,NaN,2020-12-31
4,9360002,1311969,31/12/2020,0,44P5.,1.30,NaN,2020-12-31
...,...,...,...,...,...,...,...,...
49995,46497795,2564395,31/12/2019,0,XaFqt,0.00,NaN,2019-12-31
49996,46497794,2564395,31/12/2019,0,XaIVg,0.00,NaN,2019-12-31
49997,41639202,2400982,31/12/2019,0,XaEJK,0.00,NaN,2019-12-31
49998,116371391,4900832,31/12/2019,0,X1004,0.00,NaN,2019-12-31


In [56]:
#compare 2 dates (first row and last row) from the new column

print(clinical_copy['event_dt_date_format'].iloc[-1]) # 2019
print(clinical_copy['event_dt_date_format'].iloc[0]) # 2020
clinical_copy['event_dt_date_format'].iloc[0] > clinical_copy['event_dt_date_format'].iloc[-1]

2019-12-31 00:00:00
2020-12-31 00:00:00


True

### subset 'clinical' and 'scripts' to only those patients (eid) that got the COVID-19 vaccine

In [58]:
# 39114911000001105: 'AstraZeneca', 39115611000001103: 'Pfizer',
mask_PF_AZ = scripts['dmd_code'].isin([39114911000001105, 39115611000001103]) 
scripts_vaccines_only = scripts[mask_PF_AZ] 
len(scripts[mask_PF_AZ]) # corresponds to finding above: 7 pfizer and 23 astrazeneca

30

In [59]:
# extract eids
id_vac_PF_AZ = scripts_vaccines_only['eid']
id_vac_PF_AZ

# actually not needed 

4561     5521041
4849     3747009
5499     2163004
7750     1916265
8932     5949279
9967     1304258
19612    2422930
21275    2608259
23754    3113257
29233    4568234
31157    5970140
35668    1199550
37824    5251075
38147    4443306
45729    1255920
46990    2720534
47804    1326030
54582    5667376
58686    3055374
61111    4911157
65171    1295237
70822    2080101
72345    5613818
73120    5001792
80389    5245328
87512    5040390
90802    5572579
91492    4624327
96191    1560424
99070    2193600
Name: eid, dtype: int64

In [60]:
scripts_vaccines_only.head() # contains eid and date of vaccination

,Unnamed: 0,eid,issue_date,dmd_code
4561,64814430,5521041,2021-01-20,39114911000001105
4849,39329153,3747009,2021-05-02,39115611000001103
5499,16646535,2163004,2021-01-31,39114911000001105
7750,13225838,1916265,2021-02-03,39114911000001105
8932,70996867,5949279,2021-02-22,39114911000001105


In [61]:
# subset of clinical with only vaccinated 
clinical_vaccinated_only = clinical_copy[clinical_copy['eid'].isin(id_vac)]
clinical_vaccinated_only

,Unnamed: 0,eid,event_dt,code_type,code,value,selected disease,event_dt_date_format
31073,34474547,2163004,31/12/2020,0,8HE2.,0.000000,NaN,2020-12-31
36034,47865366,2608259,31/12/2019,0,XaIAQ,36.099998,NaN,2019-12-31
36086,47865541,2608259,31/12/2019,0,242..,74.000000,NaN,2019-12-31
36117,47864684,2608259,31/12/2019,0,XaJK8,0.000000,NaN,2019-12-31
36118,47864707,2608259,31/12/2019,0,XaJKJ,0.000000,NaN,2019-12-31
36121,47864718,2608259,31/12/2019,0,XaMiI,0.000000,NaN,2019-12-31
36122,47864755,2608259,31/12/2019,0,XaMhk,0.000000,NaN,2019-12-31
36126,47864891,2608259,31/12/2019,0,XaJKn,0.000000,NaN,2019-12-31
36154,47864323,2608259,31/12/2019,0,XE0qn,0.000000,NaN,2019-12-31
36167,47864473,2608259,31/12/2019,1,Y0529,0.000000,NaN,2019-12-31


### add a column with the date of vaccination to clinical_vaccinated_only

In [62]:
# attention! only run once!
# merge clinical_copy and scripts_vaccines_only to add the issue_date (date of vaccine) to the 
clinical_vac = pd.merge(scripts_vaccines_only, clinical_copy, how ='inner', on ='eid')

In [63]:
clinical_vac

,Unnamed: 0_x,eid,issue_date,dmd_code,Unnamed: 0_y,event_dt,code_type,code,value,selected disease,event_dt_date_format
0,16646535,2163004,2021-01-31,39114911000001105,34474547,31/12/2020,0,8HE2.,0.000000,NaN,2020-12-31
1,20359621,2422930,2021-01-13,39115611000001103,42290137,31/12/2019,0,X78SS,0.000000,NaN,2019-12-31
2,20359621,2422930,2021-01-13,39115611000001103,42290412,31/12/2019,0,XaIhQ,0.000000,NaN,2019-12-31
3,23019270,2608259,2021-01-22,39114911000001105,47865366,31/12/2019,0,XaIAQ,36.099998,NaN,2019-12-31
4,23019270,2608259,2021-01-22,39114911000001105,47865541,31/12/2019,0,242..,74.000000,NaN,2019-12-31
5,23019270,2608259,2021-01-22,39114911000001105,47864684,31/12/2019,0,XaJK8,0.000000,NaN,2019-12-31
6,23019270,2608259,2021-01-22,39114911000001105,47864707,31/12/2019,0,XaJKJ,0.000000,NaN,2019-12-31
7,23019270,2608259,2021-01-22,39114911000001105,47864718,31/12/2019,0,XaMiI,0.000000,NaN,2019-12-31
8,23019270,2608259,2021-01-22,39114911000001105,47864755,31/12/2019,0,XaMhk,0.000000,NaN,2019-12-31
9,23019270,2608259,2021-01-22,39114911000001105,47864891,31/12/2019,0,XaJKn,0.000000,NaN,2019-12-31


### add a column where it says if the event (event_dt_date_format) was before or after the vaccination (issue_date)

In [64]:
# is this step really needed? seems like issue_date is in date format already.
clinical_vac['issue_date_format'] = pd.to_datetime(clinical_vac['issue_date'])
clinical_vac

,Unnamed: 0_x,eid,issue_date,dmd_code,Unnamed: 0_y,event_dt,code_type,code,value,selected disease,event_dt_date_format,issue_date_format
0,16646535,2163004,2021-01-31,39114911000001105,34474547,31/12/2020,0,8HE2.,0.000000,NaN,2020-12-31,2021-01-31
1,20359621,2422930,2021-01-13,39115611000001103,42290137,31/12/2019,0,X78SS,0.000000,NaN,2019-12-31,2021-01-13
2,20359621,2422930,2021-01-13,39115611000001103,42290412,31/12/2019,0,XaIhQ,0.000000,NaN,2019-12-31,2021-01-13
3,23019270,2608259,2021-01-22,39114911000001105,47865366,31/12/2019,0,XaIAQ,36.099998,NaN,2019-12-31,2021-01-22
4,23019270,2608259,2021-01-22,39114911000001105,47865541,31/12/2019,0,242..,74.000000,NaN,2019-12-31,2021-01-22
5,23019270,2608259,2021-01-22,39114911000001105,47864684,31/12/2019,0,XaJK8,0.000000,NaN,2019-12-31,2021-01-22
6,23019270,2608259,2021-01-22,39114911000001105,47864707,31/12/2019,0,XaJKJ,0.000000,NaN,2019-12-31,2021-01-22
7,23019270,2608259,2021-01-22,39114911000001105,47864718,31/12/2019,0,XaMiI,0.000000,NaN,2019-12-31,2021-01-22
8,23019270,2608259,2021-01-22,39114911000001105,47864755,31/12/2019,0,XaMhk,0.000000,NaN,2019-12-31,2021-01-22
9,23019270,2608259,2021-01-22,39114911000001105,47864891,31/12/2019,0,XaJKn,0.000000,NaN,2019-12-31,2021-01-22


In [55]:
clinical_vac['event_dt_date_format'].iloc[0]>=clinical_vac['issue_date_format'].iloc[0]

False

In [56]:
before_after_vaccine = []
for i in range (len(clinical_vac)):
    if clinical_vac['event_dt_date_format'].iloc[i] >= clinical_vac['issue_date_format'].iloc[i]:
        before_after_vaccine = 'after'
    else:
        before_after_vaccine = 'before'
clinical_vac['before_after_vaccine'] = before_after_vaccine

In [57]:
clinical_vac

,Unnamed: 0_x,eid,issue_date,dmd_code,Unnamed: 0_y,event_dt,code_type,code,value,disease category,event_dt_date_format,issue_date_format,before_after_vaccine
0,16646535,2163004,2021-01-31,39114911000001105,34474547,31/12/2020,0,8HE2.,0.000000,NaN,2020-12-31,2021-01-31,before
1,20359621,2422930,2021-01-13,39115611000001103,42290137,31/12/2019,0,X78SS,0.000000,NaN,2019-12-31,2021-01-13,before
2,20359621,2422930,2021-01-13,39115611000001103,42290412,31/12/2019,0,XaIhQ,0.000000,NaN,2019-12-31,2021-01-13,before
3,23019270,2608259,2021-01-22,39114911000001105,47865366,31/12/2019,0,XaIAQ,36.099998,NaN,2019-12-31,2021-01-22,before
4,23019270,2608259,2021-01-22,39114911000001105,47865541,31/12/2019,0,242..,74.000000,NaN,2019-12-31,2021-01-22,before
5,23019270,2608259,2021-01-22,39114911000001105,47864684,31/12/2019,0,XaJK8,0.000000,NaN,2019-12-31,2021-01-22,before
6,23019270,2608259,2021-01-22,39114911000001105,47864707,31/12/2019,0,XaJKJ,0.000000,NaN,2019-12-31,2021-01-22,before
7,23019270,2608259,2021-01-22,39114911000001105,47864718,31/12/2019,0,XaMiI,0.000000,NaN,2019-12-31,2021-01-22,before
8,23019270,2608259,2021-01-22,39114911000001105,47864755,31/12/2019,0,XaMhk,0.000000,NaN,2019-12-31,2021-01-22,before
9,23019270,2608259,2021-01-22,39114911000001105,47864891,31/12/2019,0,XaJKn,0.000000,NaN,2019-12-31,2021-01-22,before


# Analysing clinical events

IC10 codes are structured. The following website was used for looking up the meanings of the categories (e.g. respiratory diseases):
https://www.icd10data.com/ICD10CM/Codes

In [31]:
ic10

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17931,17931,U84.8,U848,DEFAULT,3.0,Resistance to other specified antimicrobial drug,,,,,,,
17932,17932,U84.9,U849,DEFAULT,3.0,Resistance to unspecified antimicrobial drugs,,,,,,,
17933,17933,U85,U85X,DEFAULT,3.0,Resistance to antineoplastic drugs,,,,,,,
17934,17934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


the GP_ic10 has the read code which is the CTV3. 
(However, when things get nasty, there are multiple ICD10 codes per CTV3 codes and viceversa… there is a mapping status which tells you whether the code is primary, duplicated, to revision, etc)

In [33]:
# just to see if corresponding CTV3 to IC10 works

# broadcasting for vaccinated comorbidities(CTV3) in ICD10
mask_vac_ic10 = GPic10['read_code'].isin(clinical[mask_vac]['code']) 
GPic10[mask_vac_ic10]

# mask_vac: people that got A SPECIFIC VACCINE -> TO BE CHANGED TO ANY VACCINE
# search for patients that got the vaccine in GPic10
# and look what clinical events they had via the read_code (e.g. XE01qn)


,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
76606,76606,XE0qn,R05X,D,C,C,0.0,0.0
76607,76607,XE0qn,F453,R,C,C,0.0,0.0
76608,76608,XE0qn,J111,R,C,C,0.0,0.0
76609,76609,XE0qn,J410,R,C,C,0.0,0.0
76610,76610,XE0qn,R042,R,C,C,0.0,0.0


if time: analyse specific diseases such as myocarditis etc.

In [18]:
# mask for getting codes related to myocarditis

# change to searching through 'Description' of 'ic10' 

mask_myocarditis = code['meaning'].str.contains('myocarditis', case = False)
code[mask_myocarditis]

,coding,meaning
75314,A32y1,Diphtheritic myocarditis
75808,A7423,Coxsackie myocarditis
76092,A93y1,Syphilitic myocarditis
76517,AD03.,Toxoplasma myocarditis
86230,G012.,Acute rheumatic myocarditis
86282,G1y0.,Rheumatic myocarditis
86513,G52..,Acute myocarditis
86514,G520.,Acute myocarditis in diseases EC
86515,G5200,Acute aseptic myocarditis of the newborn
86516,G5201,Acute myocarditis - coxsackie


In [19]:
# code.loc[180816, 'meaning'] # super long meaning, may leave out

In [20]:
# getting data with myocarditis
myocarditis = code[mask_myocarditis]['coding']
clinical[clinical['code'].isin(myocarditis)]
# nobody had myocarditis in this restricted data

,Unnamed: 0,eid,event_dt,code_type,code,value


In [21]:
## ishaemic heart disease
mask_ihd = code['meaning'].str.contains('ischaemic heart disease', case = False)
ischaemicHD = code[mask_ihd]['coding']
clinical[clinical['code'].isin(ischaemicHD)]

,Unnamed: 0,eid,event_dt,code_type,code,value
3603,83141410,3791841,31/12/2020,0,XE2uV,0.0
5077,46291289,2557400,31/12/2020,0,XE0oG,0.0
13147,110210389,4695920,31/12/2020,0,XE2uV,0.0
15874,25621154,1854201,31/12/2020,0,XE2uV,0.0
19336,77948891,3619074,31/12/2020,0,XE0o3,0.0
22157,119256026,4996738,31/12/2020,0,XE0o3,0.0
33646,40822178,2372522,31/12/2019,0,XE0o3,0.0
34237,21793392,1726341,31/12/2019,0,XE0oG,0.0
36998,37267699,2253583,31/12/2019,0,XE0oG,0.0
38347,7956420,1265845,31/12/2019,0,XE0oH,0.0


several comorbidities

In [22]:
# getting people with specific diseases

# new column for filling with selected disease
clinical['selected disease'] = np.nan 

# list of specific diseases, to be decided on
diseases_selec = ['ischaemic heart disease', 'cardiomyopathy', 'heart failure',\
                  'myocarditis'] 

for disease in diseases_selec:
    
    # broadcasting for selected disease
    mask = code['meaning'].str.contains(disease, case = False) 
    # broadcasting for selected disease
    sub_disease = clinical[clinical['code'].isin(code[mask]['coding'])]
    
    for i in sub_disease.index:
        # fill dataset with selected disease label
        clinical.loc[i,'selected disease'] = disease 

In [23]:
# counting how many people in the dataset have a specific disease
clinical['selected disease'].value_counts()

ischaemic heart disease    14
heart failure               1
Name: selected disease, dtype: int64

# code layout for getting categories of people w. different comorbidities

In [34]:
# code layout for getting the different categories

# subsetting into categories using IC10
# e.g. I00-I99 Diseases of the circulatory system, more details available too, e.g. Ischemic heart diseases
GPic10.dropna(inplace=True) # drop NaN to enable making mask

m_circ = GPic10['icd10_code'].str.startswith('I')
GPic10[m_circ]['read_code']

302       14A3.
303       14A4.
600       182..
602       1822.
610       1828.
          ...  
115198    Xafis
115199    Xafis
115200    Xafis
115204    Xafyq
115207    Xafyq
Name: read_code, Length: 3771, dtype: object

In [37]:
# getting corresponding CTV3 codes
m_circ_CTV3 = code['coding'].isin(GPic10[m_circ]['read_code'])
code[m_circ_CTV3]

,coding,meaning
34065,14A3.,H/O: myocardial infarct at less than 60
34066,14A4.,H/O: myocardial infarct at greater than 60
34881,182..,Chest pain
34883,1822.,Central chest pain
34889,1828.,Atypical chest pain
...,...,...
180522,XM1Qq,Thrombophlebitis
180523,XM1Qr,Superficial vein thrombosis
180524,XM1Qs,Perineal varices
180526,XM1Qu,Claudication


In [41]:
 # getting subjects with this condition
m_circ_sub = clinical['code'].isin(code[m_circ_CTV3]['coding']) # another mask

# correspond to vaccinated or not

In [45]:
# no one w. pfizer had circulatory issue before vaccine on this limited dataset
scripts[mask_AZ]['eid'].isin(clinical[m_circ_sub]['eid']).sum()

0